# Across scenario basic analysis

### Load packages

In [ ]:
import os
import sys
import urllib, io
os.getcwd()
sys.path.append("..")
sys.path.append("../utils")
sys.path.append("../analysis/utils")


import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

from tqdm.notebook import tqdm

import  matplotlib
from matplotlib import pylab, mlab, pyplot
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
plt.style.use('seaborn-white')

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')
%matplotlib inline

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [ ]:
#display all columns
pd.set_option('display.max_columns', None)

### Helper functions

In [ ]:
#helper function for pd.agg
def item(x):
    """Returns representative single item"""
    return x.tail(1).item()

### Set up directory paths to plots and data

In [ ]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir =  os.path.abspath('.')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'behavioral_experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
## add helpers to python path
if os.path.join(proj_dir,'utils') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'utils'))   

def make_dir_if_not_exists(dir_name):   
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    return dir_name

## create directories that don't already exist        
result = [make_dir_if_not_exists(x) for x in [results_dir,plot_dir,csv_dir]]

## Load in data

Assumes exported csvs from `basic_analysis.ipynb` in results folder

In [ ]:
studies = [
    "drop_pilot",
    "collision_pilot",
    "rollingsliding_pilot",
    "dominoes_pilot"
]

In [ ]:
#load all experiments as one dataframe
df = pd.concat([pd.read_csv(os.path.join(csv_dir,'trial_entries_'+l+'.csv')) for l in studies])
print("Loaded dataframes")

In [ ]:
df['study']

In [ ]:
df['scenario'] = df['study'].apply(lambda x: x.split('_')[0])

In [ ]:
df['scenario']

## Plots

### Per stimulus

In [ ]:
df['response'] = df['response'] == "YES" #encode response as boolean

In [ ]:
per_stim_agg = df.groupby('stim_ID').agg({
    'scenario' : lambda s: s.head(1),
    'correct' : lambda cs: np.mean([1 if c == True else 0 for c in cs]),
    'response' : 'mean',
    'c' : 'count',
})

In [ ]:
df['response'].value_counts()

In [ ]:
per_stim_agg

In [ ]:
g = sns.FacetGrid(per_stim_agg, col="scenario", height=6)
g.map(sns.histplot, "correct", bins=20, binrange=[0,1])

### Per person

do people have a bias for yes or no?

In [ ]:
per_person_agg = df.groupby('gameID').agg({
    'scenario' : lambda s: s.head(1),
    'correct' : lambda cs: np.mean([1 if c == True else 0 for c in cs]),
    'response' : 'mean',
    'c' : 'count',
})

What is the mean correctness for the guesses of a single participant?

In [ ]:
g = sns.FacetGrid(per_person_agg, col="scenario", height=6)
g.map(sns.histplot, "correct", bins=40, binrange=[0,1])

What is the Yes/No bias for each person? 1 corresponds to Yes. Perfect guesses every time would mean a response of 0.5

In [ ]:
g = sns.FacetGrid(per_person_agg, col="scenario", height=6)
g.map(sns.histplot, "response", bins=40, binrange=[0,1])

In [ ]:
df.groupby('scenario').agg({'response':'mean'})

### Per guess

What is the distribution of reaction times? This includes the time it takes for the 

In [ ]:
g = sns.FacetGrid(df, col="scenario", height=6)
g.map(sns.histplot, "rt", bins=40, binrange=[0,10000])

In [ ]:
df['rt'].min()